In [1]:
# @title Global Configuration
# Hyperparameters and Paths
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
DEVICE_OVERRIDE = None  # Force a device: "cuda", "mps", or "cpu"

In [ ]:
# @title Imports & Hardware Setup
import torch
import platform
import os
import sys

def get_optimal_device():
    """
    Dynamically checks for hardware resources and returns the optimal device.
    Displays specific GPU model (T4, L4, A100, etc.) and VRAM metrics.
    """
    if torch.cuda.is_available():
        device_name = torch.cuda.get_device_name(0)
        total_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        print(f"✅ NVIDIA GPU Detected: {device_name}")
        print(f"   - Total VRAM: {total_memory:.2f} GB")
        return torch.device("cuda")
    
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        print("✅ Apple Silicon (MPS) Detected.")
        return torch.device("mps")
    
    print("⚠️ No hardware acceleration found. Defaulting to CPU.")
    return torch.device("cpu")

# Initialize device
device = torch.device(DEVICE_OVERRIDE) if DEVICE_OVERRIDE else get_optimal_device()

print(f"--- Environment Summary ---")
print(f"PyTorch Version: {torch.__version__}")
print(f"Python Version: {platform.python_version()}")
print(f"System: {platform.system()} {platform.release()}")
print(f"Active Device: {device}")

✅ NVIDIA GPU Detected: Tesla T4
   - Total VRAM: 14.74 GB
--- Environment Summary ---
PyTorch Version: 2.9.0+cu126
Python Version: 3.12.12
System: Linux 6.6.105+
Active Device: cuda
